In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
df_pedidos = pd.read_excel("data/pedidos.xlsx")
df_itens = pd.read_excel("data/itens.xlsx")
df_clientes = pd.read_excel("data/clientes.xlsx") 
df_produtos = pd.read_excel("data/produtos.xlsx") 
df_reviews = pd.read_excel("data/reviews.xlsx")
df_geolocalizacao = pd.read_excel("data/geolocalizacao.xlsx") 

clientes_cluster = pd.read_excel("data/cluster_clientes.xlsx")

In [18]:
clientes = clientes_cluster.merge(df_clientes, how="inner", on="cliente_id_unico").drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
clientes_pedidos = clientes.merge(df_pedidos, how="inner", on="cliente_id")
clientes_pedidos_itens = clientes_pedidos.merge(df_itens ,how="inner", on="pedido_id")
clientes_produtos = clientes_pedidos_itens.merge(df_produtos, how="inner", on="produto_id")

clientes_reviews = clientes_pedidos.merge(df_reviews, how="inner", on="pedido_id")

In [19]:
clientes_produtos[clientes_produtos.cluster == 1].describe()

,valor_total,qnt_pedidos,cluster,cliente_CEP,numero_de_itens,preco,preco_frete,produto_comprimento_nome,produto_comprimento_descricao,produto_quantidade_fotos,produto_peso_g,produto_largura_cm,produto_altura_cm,produto_comprimento_cm
count,1381.000000,1381.000000,1381.0,1381.000000,1381.000000,1381.000000,1381.000000,1363.000000,1363.000000,1363.000000,1380.000000,1380.000000,1380.000000,1380.000000
mean,1938.153497,1.155684,1.0,43139.583635,2.317161,1026.813519,55.246387,49.450477,1461.187821,2.413059,8027.207246,40.389130,29.108696,30.947101
std,1237.964456,0.503270,0.0,30669.536406,2.641514,839.651600,54.518232,9.031492,1085.208811,1.743974,8736.558681,23.468462,19.621245,16.027739
min,1146.200000,1.000000,1.0,1020.000000,1.000000,32.000000,0.070000,14.000000,83.000000,1.000000,50.000000,14.000000,2.000000,11.000000
25%,1344.820000,1.000000,1.0,14640.000000,1.000000,219.900000,24.020000,43.000000,569.500000,1.000000,922.000000,20.000000,15.000000,18.000000
50%,1628.070000,1.000000,1.0,36033.000000,1.000000,1099.000000,38.510000,52.000000,1158.000000,2.000000,3517.500000,34.000000,22.000000,26.000000
75%,2067.420000,1.000000,1.0,74825.000000,3.000000,1460.000000,64.250000,56.000000,1973.000000,4.000000,13350.000000,53.000000,40.000000,40.000000
max,13664.080000,4.000000,1.0,99615.000000,20.000000,6735.000000,409.680000,63.000000,3992.000000,10.000000,30000.000000,105.000000,105.000000,100.000000


In [20]:
clientes_produtos[clientes_produtos.cluster == 1].produto_categoria.value_counts()

informatica_acessorios                            145
pcs                                               131
beleza_saude                                      106
automotivo                                         99
relogios_presentes                                 99
moveis_escritorio                                  91
moveis_decoracao                                   75
esporte_lazer                                      54
ferramentas_jardim                                 50
construcao_ferramentas_construcao                  44
utilidades_domesticas                              44
instrumentos_musicais                              42
bebes                                              38
eletroportateis                                    38
telefonia                                          34
cama_mesa_banho                                    30
eletrodomesticos_2                                 26
telefonia_fixa                                     24
agro_industria_e_comercio   